In [1]:
import requests
import urllib
import urllib2
import re
import json
import pandas as pd
import xml.etree.ElementTree as ET

from collections import defaultdict
from bs4 import BeautifulSoup
from scrapy import Selector
from scrapy.selector import HtmlXPathSelector
from weather import Weather

In [18]:
search_text = "Coimbatore"

<h4>Yahoo Weather Information</h4>

In [19]:
try:
    weather = Weather()
    location = weather.lookup_by_location(search_text)
    condition = location.condition()
    yahoo_data_now = ("Yahoo", int((int(condition['temp'])-32)*float(0.5556)), "", "", "")

    yahoo_url = "https://query.yahooapis.com/v1/public/yql?"
    yql_query = "select * from weather.forecast where woeid in (select woeid from geo.places(1) where text='{}')".format(search_text)
    yql_url = yahoo_url + urllib.urlencode({'q':yql_query}) + "&format=json"
    result = urllib2.urlopen(yql_url).read()
    data = json.loads(result)

    week_data = data['query']['results']["channel"]["item"]["forecast"]
    day_1 = int((int(week_data[1]['high']) - 32) * 0.5556)
    day_2 = int((int(week_data[2]['high']) - 32) * 0.5556)
    day_3 = int((int(week_data[3]['high']) - 32) * 0.5556)
    day_4 = int((int(week_data[4]['high']) - 32) * 0.5556)
    day_5 = int((int(week_data[5]['high']) - 32) * 0.5556)
    yahoo_data_now = ("Yahoo", int((int(condition['temp'])-32)*float(0.5556)), day_1, day_2, day_3, day_4, day_5)
except:
    yahoo_data_now = (
        "Yahoo",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
    )
print yahoo_data_now

('Yahoo', 27, 29, 28, 28, 28, 27)


<h4>The Weather Network</h4>

In [20]:
base_url = "https://www.theweathernetwork.com/in/api/location/search?searchText={}".format(search_text)
req = requests.get(base_url)

data = req.json()
data_url = []
if data:
    for i in data:
        if i["name"] == search_text.title() and i["country"] == "India":
            each_url = "https://www.theweathernetwork.com{}".format(i["url"])            
            data_url.append({"each_url": each_url, "code": i["code"]})

try:            
    if data_url:
    
        req_url = data_url[0]
        current_weather_url = "https://www.theweathernetwork.com/in/api/savedlocation/index/?placecodes={}".format(req_url["code"])
        current_weather_req = requests.get(current_weather_url).json()
        current_day_temp = current_weather_req[0]["temperature"]
    
        resp = urllib2.urlopen(req_url["each_url"]).read()
        soup = BeautifulSoup(resp, "lxml")
    
        current_data_day_1 = soup.find_all("div", attrs={"class": "day_1"})
        body = """{}""".format(current_data_day_1[0])
        selection_day_1 = Selector(text=body).xpath("//div").extract()
        temp_day_1 = re.findall("\d+", selection_day_1[5])[0]

        current_data_day_2 = soup.find_all("div", attrs={"class": "day_2"})
        body = """{}""".format(current_data_day_2[0])
        selection_day_2 = Selector(text=body).xpath("//div").extract()
        temp_day_2 = re.findall("\d+", selection_day_2[5])[0]

        current_data_day_3 = soup.find_all("div", attrs={"class": "day_3"})
        body = """{}""".format(current_data_day_3[0])
        selection_day_3 = Selector(text=body).xpath("//div").extract()
        temp_day_3 = re.findall("\d+", selection_day_3[5])[0]

        current_data_day_4 = soup.find_all("div", attrs={"class": "day_4"})
        body = """{}""".format(current_data_day_4[0])
        selection_day_4 = Selector(text=body).xpath("//div").extract()
        temp_day_4 = re.findall("\d+", selection_day_4[5])[0]

        current_data_day_5 = soup.find_all("div", attrs={"class": "day_5"})
        body = """{}""".format(current_data_day_5[0])
        selection_day_5 = Selector(text=body).xpath("//div").extract()
        temp_day_5 = re.findall("\d+", selection_day_5[5])[0]

        weather_network_data_now = (
            "Weather Network",
            current_day_temp,
            temp_day_1,
            temp_day_2,
            temp_day_3,
            temp_day_4,
            temp_day_5,
        )
    else:
        weather_network_data_now = (
            "Weather Network",
            "Not Found",
            "Not Found",
            "Not Found",
            "Not Found",
            "Not Found",
            "Not Found",
        )
except:
        weather_network_data_now = (
            "Weather Network",
            "Not Found",
            "Not Found",
            "Not Found",
            "Not Found",
            "Not Found",
            "Not Found",
        )
    
#url = req.json()[0]['url']
#code = req.json()[0]['code']

#ddict = defaultdict(list)

#for i in data:
#    if ddict[i["countrycode"]]:
#        ddict[i["countrycode"]].append(i)
#    else:
#        ddict[i["countrycode"]] = []
#        ddict[i["countrycode"]].append(i)

#info_urls = []        
#for k in ddict["IN"]:
#    if k["name"] == search_text.title():
#        info_urls.append(k['code'])
        
#print info_urls        
#try:
#    if info_urls:
#        data = "https://www.theweathernetwork.com/in/api/savedlocation/index/?placecodes={}".format(info_urls[0])
#        print data
#        url_req = requests.get(data)
#        print url_req.json()
#        temperature = url_req.json()[0]['temperature']
#        feels_like = url_req.json()[0]['feels_like']
#        updated = url_req.json()[0]['updatetime']
#        weather_network_data_now = ("Weather Network", str(temperature), "", "", "", "", "")
#    else:
#        weather_network_data_now = (
#            "Weather Network",
#            "Not Found",
#            "Not Found",
#            "Not Found",
#            "Not Found",
#            "Not Found",
#            "Not Found",
#        )
#except:
#    weather_network_data_now = (
#        "Weather Network",
#        "Not Found",
#        "Not Found",
#        "Not Found",
#        "Not Found",
#        "Not Foundr",
#        "Not Found",
#    )
#print weather_network_data_now

<h4> MSN Weather </h4>

In [21]:
try:
    base_url = "http://weather.service.msn.com/data.aspx?weasearchstr={}&culture=en-US&weadegreetype=F&src=outlook".format(search_text)
    req = requests.get(base_url)
    root = ET.fromstring(req.content)
    data = root.__dict__['_children'][0].__dict__['_children'][0].__dict__['attrib']
    data_1 = root.__dict__['_children'][0].__dict__["_children"][3].__dict__['attrib']['high']
    data_2 = root.__dict__['_children'][0].__dict__["_children"][4].__dict__['attrib']['high']
    data_3 = root.__dict__['_children'][0].__dict__["_children"][5].__dict__['attrib']['high']
    msn_weather_now = (
        "MSN Weather",
        int((int(data['temperature'])-32)*float(0.5556)),
        int((int(data_1)-32)*float(0.5556)),
        int((int(data_2)-32)*float(0.5556)),
        int((int(data_3)-32)*float(0.5556)),
        "",
        "",
    )
except Exception as e:
    msn_weather_now = (
        "MSN Weather",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
    )
print msn_weather_now    

('MSN Weather', 30, 31, 31, 30, '', '')


<h4> Accurate Weather </h4>

In [22]:
search_url = "https://www.accuweather.com/en/search-locations"
search_data = {"s": search_text.title()}
try:
    req = requests.post(search_url, search_data)
    resp = req.__dict__['_content']

    soup = BeautifulSoup(resp, "lxml")
    data_input_box = soup.find_all("input", attrs={'id': 's'})

    data_str = str(data_input_box[0])
    data_url = re.findall("(https://www.accuweather.com/\w+/\w+/\w+/[0-9]+/[\w-]+/[0-9]+)\"", data_str)[0]

    base_url = data_url.replace("weather-forecast", "current-weather")
    base_link_url = base_url.replace("current-weather", "daily-weather-forecast")
    base_links = ["{}?day={}".format(base_link_url, i) for i in range(1, 6)]

    req = urllib2.Request(base_url)
    read_url = urllib2.urlopen(req)
    response = read_url.read()
    soup = BeautifulSoup(response, "lxml")
    five_day_data = soup.find_all("li", attrs={'data-href': base_links})

    column_header = ('Day', 'Date', 'real_feel', 'precipitation', 'description')
    data = []
    for day in five_day_data:
        hxs = HtmlXPathSelector(day).xpath("/html/body/p/text()").extract()
        doc = """{}""".format(hxs[0].encode("utf-8"))
        each_day = re.sub("\n+",
                          ",",
                          doc.decode("unicode_escape").encode("ascii", "ignore").replace("/", "").replace("More", "")
                         )
        data.append(filter(None, tuple(each_day.split(",")))) 
    accurate_weather_now = ("Accurate Weather", data[0][2], data[1][2], data[3][2], data[4][2], "", "")
except:
    accurate_weather_now = (
        "Accurate Weather",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
        "Not Found",
    )
print accurate_weather_now

('Accurate Weather', '32', '31', '30', '30', '', '')


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:26: ScrapyDeprecationWarning: scrapy.selector.HtmlXPathSelector is deprecated, instantiate scrapy.Selector instead.


<h2>Consolidated Weather Data</h2>

In [23]:
header = ("Sites", "Current Weather", "Day_1", "Day_2", "Day_3", "Day_4", "Day_5")
dataset = [yahoo_data_now, msn_weather_now, weather_network_data_now, accurate_weather_now]
df = pd.DataFrame(data=dataset, columns=header,)
print search_text.title()+"\n"
print df.set_index("Sites")

Coimbatore

                 Current Weather Day_1 Day_2 Day_3 Day_4 Day_5
Sites                                                         
Yahoo                         27    29    28    28    28    27
MSN Weather                   30    31    31    30            
Weather Network               30    33    32    32    30    30
Accurate Weather              32    31    30    30            
